<a href="https://colab.research.google.com/github/Likelipop/DQL_Trading_based/blob/main/new_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Load Library

In [ ]:
import os
import json
from keras.models import Sequential
from keras.layers import Dense, Flatten
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

#Load data

In [ ]:
import yfinance as yf

# Tải dữ liệu cổ phiếu (ví dụ: AAPL)
df = yf.download("AAPL", start="2020-01-01", end="2023-01-01")
df['MA'] = df['Close'].rolling(window=20).mean()
df['Volume'] = df['Volume']
data = df[['Close', 'MA', 'Volume']].dropna().reset_index(drop=True)


#Def Enviroment

In [ ]:
import gym
from gym import spaces
import numpy as np
import pandas as pd

class StockTradingEnv(gym.Env):
    def __init__(self, df):
        super(StockTradingEnv, self).__init__()
        self.df = df
        self.current_step = 0

        # Giá trị khởi tạo
        self.cash = 10000  # Số tiền mặt ban đầu
        self.shares_held = 0
        self.portfolio_value = self.cash
        self.action_space = spaces.Discrete(3)  # 0: Hold, 1: Buy, 2: Sell
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(5,))

    def reset(self):
        self.current_step = 0
        self.cash = 10000
        self.shares_held = 0
        self.portfolio_value = self.cash
        return self._next_observation()

    def _next_observation(self):
        frame = np.array([
            self.df.iloc[self.current_step, 0],  # Giá hiện tại
            self.df.iloc[self.current_step, 1],  # Giá trị trung bình
            self.df.iloc[self.current_step, 2],  # Khối lượng
            self.cash,
            self.shares_held
        ])
        return frame

    def step(self, action):
        current_price = self.df.iloc[self.current_step, 0]
        self.current_step += 1

        # Xử lý hành động
        if action == 1:  # Mua
            num_shares = self.cash // current_price
            self.cash -= num_shares * current_price
            self.shares_held += num_shares
        elif action == 2:  # Bán
            self.cash += self.shares_held * current_price
            self.shares_held = 0

        # Tính giá trị danh mục
        self.portfolio_value = self.cash + self.shares_held * current_price
        reward = self.portfolio_value

        # Kiểm tra trạng thái kết thúc
        done = self.current_step >= len(self.df) - 1
        return self._next_observation(), reward, done, {}

    def render(self, mode='human'):
        print(f'Step: {self.current_step}')
        print(f'Portfolio Value: {self.portfolio_value}')


In [ ]:
# Đường dẫn file
save_path = "dqn_weights.h5"
state_path = "training_state.json"

# Load trạng thái tập huấn
if os.path.exists(state_path):
    with open(state_path, "r") as f:
        state = json.load(f)
        start_episode = state.get("current_episode", 0)
else:
    start_episode = 0

In [ ]:
# Xây dựng mô hình
def build_model(input_shape, action_space):
    model = Sequential()
    model.add(Flatten(input_shape=(1,) + input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(action_space, activation='linear'))
    return model

# Khởi tạo môi trường và mô hình
env = StockTradingEnv(data)
model = build_model(env.observation_space.shape, env.action_space.n)

# Cài đặt DQN Agent
memory = SequentialMemory(limit=50000, window_length=1)
policy = BoltzmannQPolicy()
dqn = DQNAgent(model=model, nb_actions=env.action_space.n, memory=memory, nb_steps_warmup=10, target_model_update=1e-2, policy=policy)
dqn.compile(optimizer='adam', metrics=['mae'])

# Load trọng số nếu có
if os.path.exists(save_path):
    print("Đang tải mô hình đã lưu...")
    dqn.load_weights(save_path)
    print("Mô hình đã được tải!")


In [ ]:
# Tiếp tục huấn luyện
total_episodes = 100
steps_per_episode = 200
for episode in range(start_episode, total_episodes):
    print(f"Episode {episode + 1}/{total_episodes}")

    # Huấn luyện
    dqn.fit(env, nb_steps=steps_per_episode, verbose=1)

    # Lưu trọng số
    dqn.save_weights(save_path, overwrite=True)
    print(f"Đã lưu mô hình sau Episode {episode + 1}")

    # Lưu trạng thái episode
    state = {"current_episode": episode + 1}
    with open(state_path, "w") as f:
        json.dump(state, f)